# Test notebook for LDM-503-10a milestone, test cases LVV-T1436 and LVV-T1437

This notebook performs a basic test of the ability to perform an LSST Science Platform API Aspect TAP catalog access from a user's Python process, and is set up to compare the results with the equivalent access performed via the Portal Aspect UI.

This test is in service of verifying the LDM-503-10a milestone regarding cross-Aspect authentication and authorization and its integration with TAP queries.

This notebook is intended to be executed both inside and outside the Notebook Aspect, in order to test the two ways of using the API Aspect service - with a pre-populated access token from the Notebook Aspect login process, or with an explicitly obtained token from the external get-a-token endpoint of the LSP.

## Test configuration

### User inputs required for the test

If this test is *being run outside of the Notebook Aspect* (e.g., on a home-institution system) you **MUST** have saved your API Aspect access token in the ACCESS_TOKEN environment variable before launching your local Jupyter session.  Because many ways of doing this can leave a record of your token behind, e.g., in a `.history` file, it is **strongly recommended** that you *revoke the access token* once all the LDM-503-10a tests are complete, unless you are sure you are using a secure method.  (`"export ACCESS_TOKEN; read -s ACCESS_TOKEN"` is a good start.)

URL for the TAP job for the main query executed in the Portal Aspect, obtained from clicking the "(i)" icon in the Portal table viewer toolbar.
If this is left as the empty string, the part of the test that verifies the ability to transfer a query URL from the Portal to another Aspect will be omitted.

In [1]:
portal_job_url = 'https://lsst-lsp-stable.ncsa.illinois.edu/api/tap/async/gbupxri6cznn4jt7'

It is assumed that when if you are running outside the Notebook Aspect, you will be accessing the `lsst-lsp-stable` instance of the TAP service.  This is configured here.  It is **NOT** necessary to clear this when you are running inside the Notebook Aspect.

In [2]:
off_LSP_TAP = 'https://lsst-lsp-stable.ncsa.illinois.edu/api/tap'

(If you are running _inside_ the Notebook Aspect, the code further below will use the pre-configured default TAP service for the LSP instance in which you are working, and will ignore the line above.)

Filename of the output expected from the Portal Aspect part of the LVV-C85 test cycle, LVV-T1334.  It is assumed to be placed in the working directory of your JupyterLab session.

In [3]:
portal_file = 'LVV-T1334-output.csv'

### Define the query parameters

In [4]:
# This is the AllWISE Source Catalog (the equivalent of LSST `Object`)
allwise_table = 'wise_00.allwise_p3as_psd'
# The equatorial coordinate variables are named "ra" and "decl" in the LSST PDAC copy.
(allwise_ra, allwise_dec) = ('ra', 'decl')

In [5]:
# Cone search parameters; must match LVV-T1334 narrative.
center = ( 2, 0 )  # RA, dec point, in degrees
radius = 0.5       # radius of cone in degrees

In [6]:
# Upper limit on query size
maxrows = 50000    # actual query should return more like 12,000 rows

---

## Preset test-passed statuses

In [7]:
# Was the query from this notebook executed successfully?
test_performed_query = False

# Was the comparison with the output from the Portal test executed successfully?
test_compared_portal = False if portal_file != None and portal_file != ''       else None

# Was the comparison with data from the Portal's TAP job endpoint executed successfully?
test_compared_job = False    if portal_job_url != None and portal_job_url != '' else None

## Obtain the PyVO TAPQuery object for executing the query

### Set things up correctly for running either internally or externally.

In [8]:
import os

In [9]:
try:
    ext_tap = os.environ['EXTERNAL_TAP_URL']
except KeyError:
    ext_tap = ''

try:
    ext_inst = os.environ['EXTERNAL_INSTANCE_URL']
except KeyError:
    ext_inst = ''

In [10]:
( ext_tap, ext_inst )

('', '')

In [11]:
if ext_tap == '' and ext_inst == '':
    # We must be running outside the Notebook Aspect.  Push the pre-configured external
    # TAP access URL into the environment so that the PyVO-helper code will pick it up.
    os.environ['EXTERNAL_TAP_URL'] = off_LSP_TAP

### Get the "helper utility" that populates the query service with authorization data.

In [12]:
from astropy.table import Table

In [13]:
from jupyterlabutils.notebook import get_catalog, retrieve_query

In [14]:
service = get_catalog()

### Construct the query

In [15]:
adql_query = "SELECT ra, decl, cntr, source_id, coadd_id, w1mpro, w2mpro, w3mpro, w4mpro FROM " + allwise_table + \
    " WHERE CONTAINS(POINT('ICRS'," + allwise_ra + "," + allwise_dec + ")," + \
                    f"CIRCLE('ICRS', {center[0]}, {center[1]}, {radius} ))=1"

In [16]:
adql_query

"SELECT ra, decl, cntr, source_id, coadd_id, w1mpro, w2mpro, w3mpro, w4mpro FROM wise_00.allwise_p3as_psd WHERE CONTAINS(POINT('ICRS',ra,decl),CIRCLE('ICRS', 2, 0, 0.5 ))=1"

In [17]:
results = service.search(adql_query)

### Examine the results

In [18]:
query_table = results.to_table()
query_rows = len(query_table)
query_rows

12717

Show that the rows refer to unique objects

In [19]:
query_id_set = set(query_table['cntr'])
query_key_count = len(query_id_set)
query_unique_keys = ( query_key_count == query_rows )
query_unique_keys

True

Roughly verify the cone search did what it was supposed to.

In [20]:
from numpy import hypot

In [21]:
raoff = query_table[allwise_ra]-center[0]
decoff = query_table[allwise_dec]-center[1]
radoff = hypot(raoff,decoff)

In [22]:
( min(raoff), max(raoff), min(decoff), max(decoff), min(radoff), max(radoff) )

(-0.4993004999999999,
 0.4997617000000001,
 -0.4994012,
 0.4980328,
 0.0050964593454670285,
 0.49999259862406964)

In [23]:
delta = 0.01
test_query_bounds = ( 10000 < query_rows < 15000 ) and \
    ( -radius         <= min(raoff)  <= -radius + delta ) and \
    (  radius - delta <= max(raoff)  <= radius ) and \
    ( -radius         <= min(decoff) <= -radius + delta ) and \
    (  radius - delta <= max(decoff) <= radius ) and \
    (  0              <= min(radoff) <= delta ) and \
    (  radius - delta <= max(radoff) <= radius )
test_query_bounds

True

In [24]:
test_performed_query = ( query_unique_keys and test_query_bounds )
test_performed_query

True

Add an index to the result to facilitate analysis

In [25]:
query_table.add_index('cntr')

## Define a simple table comparison function

In [26]:
def compare_tables( ta, tb ):
    # Determine whether the two tables have the same keys:
    ta_id_set = set(ta['cntr'])
    tb_id_set = set(tb['cntr'])
    same_keys = (ta_id_set == tb_id_set)
    same_values = False
    if same_keys:
        # Requires indexes to have been set up on the two tables:
        diff_rows = [(cntr,
                      abs(ta.loc[cntr]['ra']     - tb.loc[cntr]['ra']),
                      abs(ta.loc[cntr]['decl']   - tb.loc[cntr]['decl']),
                      abs(ta.loc[cntr]['w1mpro'] - tb.loc[cntr]['w1mpro'])) 
                     for cntr in query_id_set]
        deviation_max = max([row[1] for row in diff_rows]) + \
                        max([row[2] for row in diff_rows]) + \
                        max([row[3] for row in diff_rows])
        same_values = ( deviation_max == 0 )
    return ( same_keys, same_values )

## Compare the query results with those obtained from the Portal Aspect UI

This step tests that the Portal query can be reproduced by the equivalent query through the Python interface.  This is in a way obvious just from the design, since both queries go through the same TAP service - and because the ADQL from the Portal query is available for inspection.  However, it does follow two somewhat different paths in detail, and tests some issues associated with the conversions between text and numeric forms of the query results.

In [27]:
if portal_file:
    portal_table = Table.read(portal_file, format='csv')
    print( 'The table saved in the Portal has', len(portal_table), 'rows.' )
    # Add an index to the result to facilitate analysis
    portal_table.add_index('cntr')
    # Compare the tables
    portal_same = compare_tables(portal_table, query_table)
    print( 'Results from comparison with live query:', portal_same )
    test_compared_portal = portal_same[0] and portal_same[1]

The table saved in the Portal has 12717 rows.
Results from comparison with live query: (True, True)


## Compare the query results with those obtained from the TAP job URL for the Portal Aspect query

This step verifies that a query performed in the Portal Aspect can be accessed directly from the Notebook Aspect, or externally through the API Aspect, simply by transferring a URL reference.

In [28]:
if portal_job_url:
    portal_job = retrieve_query( portal_job_url )
    portal_job_results = portal_job.fetch_result()
    portal_job_table = portal_job_results.to_table()
    print( 'The table retrieved from the TAP job URL from the Portal has', len(portal_job_table), 'rows.' )
    # Add an index to the result to facilitate analysis
    portal_job_table.add_index('cntr')
    # Compare the tables
    portal_job_same = compare_tables(portal_job_table, query_table)
    print( 'Results from comparison with live query:', portal_job_same )
    test_compared_job = portal_job_same[0] and portal_job_same[1]

The table retrieved from the TAP job URL from the Portal has 12717 rows.
Results from comparison with live query: (True, True)


## Report results

In [29]:
print( 'Successfully completed query from notebook:', test_performed_query )
print( 'Successful comparison to Portal query output:', test_compared_portal )
print( 'Successful comparison to TAP job from Portal query:', test_compared_job )

Successfully completed query from notebook: True
Successful comparison to Portal query output: True
Successful comparison to TAP job from Portal query: True
